In [13]:
# imports for training data
!if ( ! ls . | grep pytransform ); then git clone https://github.com/mhask94/pytransform.git; fi
# !git clone https://github.com/mhask94/pytransform.git
from pytransform.common import skew
from pytransform.quaternion import Quaternion as Quat
import numpy as np

pytransform


In [0]:
# imports for pytorch
import torch
import torch.nn as nn

In [0]:
# classes for quadrotor state and dynamics
class State():
    def __init__(self, arr=np.empty(0)):
        if len(arr) == 0:
            self.arr = np.zeros((10,1), dtype=np.float64)
            self.arr[3] = 1
        else:
            assert arr.shape == (10, 1)
            if not arr.dtype == np.float64:
              arr = np.array(arr, dtype=np.float64)
            arr.dtype = np.float64
            self.arr = arr

    def __getitem__(self, position):
        return self.arr[position]
    def __str__(self):
        s = 'p: ' + str(self.p.flatten()) + '\nq: ' + self.q.__str__() + \
                '\nv: ' + str(self.v.flatten())
        s = s.replace('[ ', '[')
        s = s.replace(', ', ' ')
        s = s.replace(' ]', ']')
        return s
    def __repr__(self):
        return self.__str__()
    def __add__(self, other):
        assert other.shape == (9, 1)
        out = np.empty(self.arr.shape)
        out[:3]  = self.p + other[:3]
        out[3:7] = (self.q + other[3:6]).elements
        out[7:]  = self.v + other[6:]
        return State(out)
    def __iadd__(self, other):
        assert other.shape == (9, 1)
        self.arr[:3] += other[:3]
        self.arr[3:7] = (self.q + other[3:6]).elements
        self.arr[7:] += other[6:]
        return self
    def __sub__(self, other):
        assert isinstance(other, State)
        dx = np.empty((9,1))
        dx[:3]  = self.p - other.p
        dx[3:6] = self.q - other.q
        dx[6:]  = self.v - other.v
        return dx
    @property
    def p(self):
        return self.arr[:3]
    @property
    def q(self):
        return Quat(self.arr[3:7])
    @property
    def v(self):
        return self.arr[7:]
    @property
    def elements(self):
        return self.arr
    @property
    def X(self):
        return Xform(self.arr[:7])
    def copy(self):
        return State(self.arr.copy())

class Dynamics():
    def __init__(self, x0=np.empty(0)):
        self.x = State(x0)
        self.k1 = np.zeros((9,1))
        self.k2 = np.zeros((9,1))
        self.k3 = np.zeros((9,1))
        self.k4 = np.zeros((9,1))
        self.cd = 0.1
        e_z = np.array([[0,0,1]]).T
        self.g = 9.8065 * e_z
        self.se = 0.5

    def run(self, u, dt):
        self.k1 = self.f(self.x, u)
        self.k2 = self.f(self.x + self.k1*(dt/2), u)
        self.k3 = self.f(self.x + self.k2*(dt/2), u)
        self.k4 = self.f(self.x + self.k3*dt, u)
        self.x += (self.k1 + 2*(self.k2 + self.k3) + self.k4) * (dt/6)

    def f(self, x, u):
        s, w = u[0], u[1:]

        dx = np.empty(self.k1.shape)
        dx[:3] = x.q.rota(x.v)
        dx[3:6] = w
        dx[6:] = -self.g*(s/self.se) - self.cd*x.v + x.q.rotp(self.g) - \
                skew(w) @ x.v
        return dx

    @property
    def state(self):
        return self.x


In [5]:
x = State()
dx = np.array([[1,2,3,0.1,0,0,.1,.2,.3]]).T

y = x + dx
print(y)

p: [1. 2. 3.]
q: [0.9987502603949663 0.04997916927067834i 0.0j 0.0k]
v: [0.1 0.2 0.3]


In [16]:
def testConvTransposeSize():
  x_test = torch.zeros(1,1,10,1)
  up = nn.ConvTranspose2d(1, 1, (2,20), padding=0, stride=2)
  up_test = up(x_test)
  print('up: ', up_test.size())

  down = nn.Conv2d(1,1, (2,20), padding=0, stride=2)
  down_test = down(up_test)
  print('down: ', down_test.size())

testConvTransposeSize()

up:  torch.Size([1, 1, 20, 20])
down:  torch.Size([1, 1, 10, 1])


In [0]:
class UBlock(nn.Module):
  def __init__(self, c_in, c_out, after='none'):
    super(UBlock, self).__init__()
    self.main_net = nn.Sequential(
        nn.Conv2d(c_in, c_out, kernel=(3,3), padding=(1,1)),
        nn.ReLU(),
        nn.Conv2d(c_out, c_out, kernel=(3,3), padding=(1,1)),
        nn.ReLU,
    )
    self.after = after
    if after == 'up':
      self.after_net = nn.ConvTranspose2d(c_out, c_out//2, (2,2), padding=0, stride=2)
    elif after == 'down':
      self.after_net = nn.Conv2d(c_out, c_out, kernel=(3,3), padding=(1,1), stride=2)
    elif after == 'end':
      self.after_net = nn.Conv2d(c_out, 1, (1,1), padding=0)
    else: # none
      self.after_net = None

  def forward(self, x):
    main_out = self.main_net(x).squeeze(2).squeeze(2)
    after_out = self.after_net(main_out).squeeze(2).squeeze(2)
    if self.after == 'up':
      return main_out, after_out
    else:
      return after_out

class DynamicsNN(nn.Module):
  def __init__(self):
    super(DynamicsNN, self).__init__()
    self.beg = nn.ConvTranspose2d(1, 1, (2,20), padding=0, stride=2)
    self.up1 = UBlock(1, 512, after='up')
    self.up2 = UBlock(256, 256, after='up')
    self.up3 = UBlock(128, 128, after='up')
    self.up4 = UBlock(64, 64, after='up')
